In [1]:
import os
import sys

sys.path.append('../..')

from seismicpro.batchflow import Dataset, DatasetIndex, Pipeline, V, B, L, I, W, P, R, D

from seismicpro.src import SeismicBatch, SeismicDataset, FieldIndex, merge_segy_files

In [2]:
path_raw = '../../../data/PREP/0825_50Hz_45Hz_fnum/4_PREP_input_50Hz_45Hz_fnum_0825.sgy'
path_lift = '../../../data/PREP/0825_50Hz_45Hz_fnum/4_PREP_output_50Hz_45Hz_fnum_0825.sgy'

test_path_raw = '../../../data/PREP/0815_50Hz/6_input_50h.sgy'
test_path_lift = '../../../data/PREP/0815_50Hz/6_output_50h.sgy'

tmp_dump_path = './tmp/'

paths = [path_raw, path_lift, test_path_raw, test_path_lift]

for p in paths:
    # Clean the tmp directory
    filelist = os.listdir(tmp_dump_path)
    for f in filelist:
        os.remove(os.path.join(tmp_dump_path, f))
    # Generate index for sgy file
    index_train = (FieldIndex(name='tmp', 
                    extra_headers='all',
                    path=p))
    # Create dataset of shot gathers
    eq_dtst = SeismicDataset(index_train)
    # Run pipeline to calculate equalization parameters
    preproc_ppl = (
        eq_dtst.pipeline()
            .load(fmt='sgy', components='tmp')
            .find_equalization_params(B(), component='tmp', survey_id_col='YearDataRecorded', container_name='equal_params')
            .run(batch_size=8, n_epochs=1, shuffle=False, drop_last=False, bar=True))
    # Run pipeline to apply equalization and save resulting data
    eq_ppl = (
        eq_dtst.pipeline()
            .load(fmt='sgy', components='tmp')
            .equalize(src='tmp', dst='tmp', params=D('equal_params'))
            .dump(path=L(lambda x: os.path.join(tmp_dump_path, str(x) + '.sgy'))(I()),
                  src='tmp', fmt='segy', split=False)
            .run_later(batch_size=8, n_epochs=1, shuffle=False, drop_last=False, bar=True))
    # Merge batch files into one nice sgy file
    output_path = os.path.splitext(p)[0] + '_eq.sgy'
    eq_ppl.after.merge_segy_files(output_path=output_path, extra_headers='all', path=os.path.join(tmp_dump_path, '*.sgy'))
    
    eq_ppl.run()

  0%|          | 0/4 [00:00<?, ?it/s]../../seismicpro/batchflow/batchflow/batch.py:297: UserWarning: All batch data is erased
  warnings.warn("All batch data is erased")
100%|██████████| 2/2 [00:02<00:00,  1.37s/it]


In [4]:
eq_dtst.equal_params

{2010: 2.8872700683039585,
 99: 3.935645536837002,
 'survey_id_col': 'YearDataRecorded'}